In [ ]:
import time
from collections import namedtuple
try:
    from skimage import io as skimage_io
except:
    !pip install scikit-image
    from skimage import io as skimage_io
from self_fourier_shell_correlation import fsc_utils as fsc
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# apt install cm-super-minimal
# apt install dvipng
plt.rcParams.update({
    "text.usetex": True,
    #"font.family": "Helvetica",
    "font.family": "Serif",
    "text.latex.preamble": r"\usepackage{amsmath} \usepackage{amsfonts}"
})

In [ ]:
Args = namedtuple("args", ["input", "input1", "input2", "input3", "input4"])
args = Args("http://www.hpca.ual.es/~vruiz/images/FDM/TwoPhoton_BPAE_R.png",
            "http://www.hpca.ual.es/~vruiz/images/FDM/TwoPhoton_BPAE_R_1.png",
            "http://www.hpca.ual.es/~vruiz/images/FDM/TwoPhoton_BPAE_R_2.png",
            "http://www.hpca.ual.es/~vruiz/images/FDM/TwoPhoton_BPAE_R_3.png",
            "http://www.hpca.ual.es/~vruiz/images/FDM/TwoPhoton_BPAE_R_4.png"
           )

In [ ]:
X = skimage_io.imread(args.input)  # Ground Truth
X1 = skimage_io.imread(args.input1)  # Noisy instance 1
X2 = skimage_io.imread(args.input2)  # Noisy instance 2
X3 = skimage_io.imread(args.input3)  # Noisy instance 3
X4 = skimage_io.imread(args.input4)  # Noisy instance 4

In [ ]:
SCBS_freq, SCBS_c_avg = fsc.get_SFRC_curve__subsampled_chessboard(X1)
len(SCBS_freq)

In [ ]:
r = X1.shape[0] // 2
voxel = 1
freq = fsc.get_radial_spatial_frequencies(X1, voxel, mode='full')
sfsc = np.mean(fsc.single_image_frc(X1, r), axis=0)
sfsc_adjusted = 2*sfsc / (1 + sfsc)
_, c_avg = fsc.get_SFRC_curve__even_odd(X1)

In [ ]:
cmap_a = [plt.get_cmap('tab20c').colors[idx] for idx in [0, 4, 8, 12]]
cmap_b = [plt.get_cmap('tab20c').colors[idx] for idx in [2, 6, 10, 12]]
plt.figure(figsize=(5,4))
plt.plot(freq, sfsc, linewidth=2, label='SFSC')
plt.plot(freq, sfsc_adjusted, linewidth=2, label='SFSC (adjusted)')
plt.plot(freq, c_avg, linewidth=1, label='?')
plt.rc('axes', labelsize=12)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)
plt.rc('legend', fontsize=10)
plt.grid(which='major', linestyle='--')
plt.xlabel('spatial frequency 'r'(${\AA}^{-1}$)', labelpad=-1.5)
plt.title('Fourier shell correlation')
plt.legend(loc='lower left')
# plt.savefig('')
plt.show()

In [ ]:
EOS_freq, EOS_c_avg = fsc.get_SFRC_curve__even_odd(X1)
print(len(EOS_freq), len(EOS_c_avg))

In [ ]:
CBS_freq, CBS_c_avg = fsc.get_SFRC_curve__chessboard(X1)
print(len(CBS_freq), len(CBS_c_avg))

In [ ]:
ICBS_freq, ICBS_c_avg = fsc.get_SFRC_curve__interpolated_chessboard(X1)
print(len(ICBS_freq), len(ICBS_c_avg))

In [ ]:
SCBS_freq, SCBS_c_avg = fsc.get_SFRC_curve__subsampled_chessboard(X1)
print(len(freq), len(SCBS_c_avg))

In [ ]:
FRC_freq, FRC_c_avg_1 = fsc.get_FRC_curve__square_imgs(X1, X2)
FRC_freq, FRC_c_avg_2 = fsc.get_FRC_curve__square_imgs(X3, X4)
FRC_c_avg = (FRC_c_avg_1 + FRC_c_avg_2) / 2

In [ ]:
SPRS_freq, SPRS_c_avg = fsc.get_SFRC_curve__SPRS(image=X1, std_dev=2.0, sigma_poly=1.0, window_side=5)
print(len(SPRS_freq), len(SPRS_c_avg))

In [ ]:
plt.plot(EOS_freq, EOS_c_avg, linewidth=1, label="EOS")
plt.plot(CBS_freq, CBS_c_avg, linewidth=1, label="CBS")
plt.plot(ICBS_freq, ICBS_c_avg, linewidth=1, label="ICBS")
plt.plot(SCBS_freq, SCBS_c_avg, linewidth=1, label="SCBS")
plt.plot(SPRS_freq, SPRS_c_avg, linewidth=1, label="SPRS")
plt.plot(freq, FRC_c_avg, linewidth=2, label="True FRC")
plt.grid(which="major", linestyle="--")
plt.xlabel(r"$f/f_s$")
plt.ylabel(r"\text{SFRC}")
plt.title(r"$\mathrm{TwoPhoton\_BPAE\_R\_noisy}$")
plt.legend()
plt.savefig('TwoPhoton_BPAE_R_noisy_SFRC.pdf', bbox_inches='tight')
plt.show()

In [ ]:
import pandas as pd  # For easy rolling mean calculation
def smooth_curve(data, window_size):
    series = pd.Series(data)
    smoothed = series.rolling(window=window_size, center=True).mean()
    # Handle NaN values at the edges by filling with the original values
    smoothed = smoothed.fillna(series)
    return smoothed.to_numpy()

window = 5  # Adjust this value to control the smoothing strength

In [ ]:
plt.plot(EOS_freq, smooth_curve(EOS_c_avg, window), linewidth=1, label="EOS")
plt.plot(CBS_freq, smooth_curve(CBS_c_avg, window), linewidth=1, label="CBS")
plt.plot(ICBS_freq, smooth_curve(ICBS_c_avg, window), linewidth=1, label="ICBS")
plt.plot(SCBS_freq, smooth_curve(SCBS_c_avg, window), linewidth=1, label="SCBS")
plt.plot(SPRS_freq, smooth_curve(SPRS_c_avg, window), linewidth=1, label="SPRS")
plt.plot(FRC_freq, smooth_curve(FRC_c_avg, window), linewidth=2, label="True FRC")
plt.grid(which="major", linestyle="--")
plt.xlabel(r"$f/f_s$")
plt.ylabel(r"$\text{SFRC (smoothed)}$")
plt.title(r"$\mathrm{TwoPhoton\_BPAE\_R\_noisy}$")
plt.legend()
plt.savefig('TwoPhoton_BPAE_R_noisy_SFRC.pdf', bbox_inches='tight')
plt.show()